In [1]:
import ee
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
# Autentica com o Google Earth Engine
# Descomente se for a primeira execução
# ee.Authenticate()


In [4]:

# Inicializa o Earth Engine
ee.Initialize(project=os.environ.get("GOOGLE_EARTH_ENGINE_PROJECT_ID"))


In [5]:
# Carrega geometria de Minas Gerais (FAO GAUL, nível estadual)
mg = (
    ee.FeatureCollection("FAO/GAUL/2015/level1")
    .filter(ee.Filter.eq("ADM1_NAME", "Minas Gerais"))
    .filter(ee.Filter.eq("ADM0_NAME", "Brazil"))
    .geometry()
)

In [6]:
# Seleciona a coleção NDVI do MODIS (250m, 16 dias)
ndvi_collection = ee.ImageCollection("MODIS/006/MOD13Q1") \
    .filterBounds(mg) \
    .filterDate("2010-01-01", "2023-12-31") \
    .select('NDVI')

/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


In [7]:
# Função para reduzir por média regional e transformar em Feature com data e valor
def image_to_feature(img):
    mean_dict = img.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=mg, scale=250, maxPixels=1e13
    )
    return ee.Feature(
        None, {"date": img.date().format("YYYY-MM-dd"), "ndvi": mean_dict.get("NDVI")}
    )

In [8]:
# Aplica a função a todas as imagens
ndvi_features = ndvi_collection.map(image_to_feature)

In [9]:
# Coleta os dados em Python
ndvi_list = ndvi_features.getInfo()['features']

In [12]:
import pandas as pd  # Converte para DataFrame


data = [
    {"date": f["properties"]["date"], "ndvi": f["properties"]["ndvi"]}
    for f in ndvi_list
]
df = pd.DataFrame(data)
df

,date,ndvi
0,2010-01-01,6961.220523
1,2010-01-17,6836.056335
2,2010-02-02,6652.292193
3,2010-02-18,6482.417253
4,2010-03-06,7004.443686
...,...,...
297,2022-12-03,6368.406609
298,2022-12-19,6033.379263
299,2023-01-01,6433.541609
300,2023-01-17,7135.002445


In [11]:
# Exporta como CSV
if not os.path.exists("../data/GOOGLE_EARTH_ENGINE"):
    os.makedirs("../data/GOOGLE_EARTH_ENGINE")
df.to_csv("../data/GOOGLE_EARTH_ENGINE/ndvi_minas_gerais.csv", index=False)
print("✅ CSV salvo com sucesso!")

✅ CSV salvo com sucesso!
